In [1]:
import pandas as pd

url = 'https://drive.google.com/uc?id=1eqFIZCJTkXY-PsTeS-uz_DousIH3XQu-'  
rides = pd.read_csv(url) # Открываем rides_info - информация про поездки
rides.sample(5)

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
534404,r17989452S,l29142213W,m1U,2020-01-27,6.64,48,474,62,122.422641,0,2789.009550,0,-6.416323,2.301
177213,a14198824D,M20404037C,A1i,2020-02-12,4.97,5,75,39,66.000000,1,164.625185,0,9.925237,34.262
467618,s12770196V,h-1179139J,z1f,2020-02-12,5.40,12,152,48,63.000000,0,469.241250,0,-2.830689,-13.926
106550,N19021616c,H16125822e,j1i,2020-02-02,2.90,20,274,39,94.000000,1,434.684334,0,-6.786111,29.794
525562,i21370499e,l-1177682G,Z1H,2020-02-13,4.17,5,94,51,123.044404,0,275.221538,0,0.390967,-7.046


In [3]:
# Определяем lambda функцию с тернарным оператором внутри
my_awesome_function = lambda x: 'more_30' if x > 30 else 'no_more_30'

# Применяем функцию к столбцу
rides['speed_avg_greater_30'] = rides['speed_avg'].apply(my_awesome_function)

# Смотрим результат
rides['speed_avg_greater_30'].value_counts() 

more_30       699420
no_more_30     40080
Name: speed_avg_greater_30, dtype: int64

Если рейтинг выше 6 и максимальная скорость поездки меньше 120 км/ч, то стоимость поездки (ride_cost) уменьшите на 5%.

В противном случае: стоимость поездки (ride_cost) увеличьте на 5%.



In [6]:
def price_making(x):
    if x['rating'] > 6 and x['speed_max'] < 120:
        return x['ride_cost'] * 0.95
    else:
        return x['ride_cost'] * 1.05

df['final_ride_cost'] = df.apply(price_making, axis=1).astype(int)

In [16]:
cols = ["rating", "ride_duration", "ride_cost"]

# Производим группировку
rides_gr = rides.groupby("car_id", as_index=False)[cols].mean()

# Смотрим результаты
rides_gr.head()

,car_id,rating,ride_duration,ride_cost
0,A-1049127W,4.255172,1289.034483,15201.436782
1,A-1079539w,4.088046,2148.810345,27007.477011
2,A-1162143G,4.662299,479.017241,4599.678161
3,A-1228282M,4.225172,2167.534483,27379.522989
4,A-1339912r,4.690115,1436.614943,16185.545977


In [17]:
cols = ["rating", "ride_cost"]

# Производим группировку
rides_info_gr = rides.groupby("car_id", as_index=False)[cols].agg(["count", "mean"])


In [21]:

# Убираем мультииндекс
rides_info_gr.columns = ["_".join(c) for c in rides_info_gr.columns]

# Смотрим результаты (car_id теперь не столбец, а индекс)
rides_info_gr.head()

,rating_count,rating_mean,ride_cost_count,ride_cost_mean
car_id,,,,
A-1049127W,174,4.255172,174,15201.436782
A-1079539w,174,4.088046,174,27007.477011
A-1162143G,174,4.662299,174,4599.678161
A-1228282M,174,4.225172,174,27379.522989
A-1339912r,174,4.690115,174,16185.545977


    Столбец feature_1 - Количество уникальных значений остановок (stop_times).
    Столбец feature_2 - Второй максимум столбца максимальной скорости (speed_max).
    Столбец feature_3 - Отношение минимального и максимального значения рейтинга.
    Столбец feature_4 - Самое популярное значение числа остановок (другим словами, мода).
    Столбец feature_5 - Стандартное отклонение стоимостей поездок (ride_cost). 

In [22]:
import numpy as np
result = df.groupby('car_id', as_index=False).agg(
    feature_1 = ('stop_times', 'nunique'),
    feature_2 = ('speed_max', lambda x: x.sort_values().iloc[-2]),
    feature_3 = ('rating', lambda x: x.min() / x.max()),
    feature_4 = ('stop_times', lambda x: x.mode()[0]),
    feature_5 = ('ride_cost', 'std')
)

In [24]:
result.sample(2)

,car_id,feature_1,feature_2,feature_3,feature_4,feature_5
411,F-1561156R,20,116.0,0.010929,0,73810.355588
4026,x-5000537P,4,NaN,0.000000,1,81807.023823


Датасет rides содержит столбцы: car_id, ride_id, rating, ride_duration, ride_cost

Датасет cars содержит столбцы:car_id, model, riders, target_class

    Обогатите датасет rides про поездки информацией о будущем виде поломки, используя датасет cars. 
    Удалите лишние столбцы. Таблицу rides надо дополнить лишь столбцом target_class.
    Удалите поездки, для которых нет информации про поломки.  
    Запишите в переменную result.

In [25]:
# rides = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_4_rides.csv')
cars = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_4_cars.csv')


In [30]:
rides.merge(cars[['car_id','target_class']], on='car_id', how='inner')

    Обогатите датасет rides про поездки дополнительными признаками без использования метода merge.
    Добавьте столбец car_ride_count - общее число поездок для этой машины.
    Добавьте столбец user_rating_max - максимальный рейтинг, поставленный этим пользователем.
    Добавьте столбец user_ride_cost_mean - средняя стоимость поездки этого пользователя.
    Результат запишите в переменную result.

In [38]:
rides['car_ride_count'] = rides.groupby('car_id')['rating'].transform('count')
rides['user_rating_max'] = rides.groupby('user_id')['rating'].transform('max')
rides['user_ride_cost_mean'] = rides.groupby('user_id')['ride_cost'].transform('mean')

    Обогатите датасет rides про поездки дополнительными признаками без использования метода merge.
    🤖 Добавьте столбец feature_1 — 75% перцентиль столбца ride_cost для каждого пользователя (user_id).
    🎃 Добавьте столбец feature_2 — разницу максимального и минимального значения rating, деленную на среднее столбца rating для каждого пользователя (user_id).
    👽 Добавьте столбец feature_3  — количество значений ride_duration, которое больше 40 для каждой машины (car_id).
    👾 Добавьте столбец feature_4 — сумму значений ride_duration, которая больше 40 для каждой машины (car_id).
    Результат запишите в переменную result.

In [64]:
rides = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_6.csv')


In [68]:
(rides['ride_duration'] > 40)

0      True
1      True
2      True
3      True
4      True
      ...  
95     True
96    False
97    False
98     True
99    False
Name: ride_duration, Length: 100, dtype: bool

In [69]:
rides['feature_1'] = rides.groupby('user_id')['ride_cost'].transform(lambda x: np.percentile(x, 75))
rides['feature_2'] = rides.groupby('user_id')['rating'].transform(lambda x: (x.max() - x.min()) / x.mean())
rides['feature_3'] = rides.groupby('car_id')['ride_duration'].transform(lambda x: (x > 40).sum())
rides['feature_4'] = rides.groupby('car_id')['ride_duration'].transform(lambda x: x[x > 40].sum())

Индексом во всех таблицах будет user_id.

    🙂 Первая таблица: Сколько раз пользователь брал машины определенного класса? 
    🙃 Вторая таблица: Какими были минимальная, максимальная, средняя дистанция и стоимость поездки пользователя?
    🙂 Третья таблица: Сколько уникальных машин брал пользователь в аренду?
    Далее соединяем по ключу (смерживаем) все 3 таблицы в одну.
    И в конце добавляем бинарный таргет в столбец crazy_driver: Является ли наш пользователь безумным гонщиком? Если его максимальная скорость превышала 100 км/ч, то 1; если не превышала, то 0.

Результат решения запишите в переменную result.

In [147]:
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_7.csv')


In [148]:
df.sample(2)

,user_id,car_type,car_id,distance,ride_cost,speed_max,model
24,S51575404w,economy,A-1049127W,398.859961,257,69.0,Hyundai Solaris
16,T73601037O,economy,A-1049127W,121.231506,46,83.0,Hyundai Solaris


In [160]:
df1 = pd.pivot_table(
    df, index='user_id', columns='car_type', values='car_id', aggfunc='count', fill_value=0, 
)
df1.columns = [f'{i}_count' for i in df1.columns]

df2 = pd.pivot_table(
    df, index='user_id', values=['distance', 'ride_cost'], aggfunc=['min', 'max', 'mean'],  
)
df2.columns = [f'{i[1]}_{i[0]}' for i in df2.columns]

df3 = pd.pivot_table(
    df, index='user_id', values=['car_id'], aggfunc=lambda x: x.nunique()
)
df3.columns = [f'{i}_nunique' for i in df3.columns]

df4 = pd.pivot_table(
    df, index='user_id', values=['speed_max'], aggfunc=lambda x: int(x.max() > 100)
)
df4.columns = ['crazy_driver']
result = df1.merge(df2, on='user_id').merge(df3, on='user_id').merge(df4, on='user_id')
result = result.reset_index()
result[['economy_count',	'standart_count']] = result[['economy_count',	'standart_count']].astype(float)


In [157]:
# В ans записан правильный ответ
ans = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_7_ans.csv')